In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy
import tensorflow as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [7]:
def stockPredictionModel(name_of_stock, number_of_share, start_date, end_date):
    df = pd.read_csv('stock_nifty50.csv')
    df.head()
    df["Date"] = pd.to_datetime(df["Date"])
    # name_of_stock = input("Enter Name Of Stock")
    # number_of_share = int(input("Number Of Share"))
    Y = df[df["Symbol"]==name_of_stock][["Close"]]
    # start_date = input("Start Date")
    # end_date = input("End Date")
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)



    scaler=MinMaxScaler(feature_range=(0,1))
    Y=scaler.fit_transform(np.array(Y).reshape(-1,1))



    training_size=int(len(Y)*0.65)
    test_size=len(Y)-training_size
    train_data,test_data=Y[0:training_size,:],Y[training_size:len(Y),:1]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, time_step):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return numpy.array(dataX), numpy.array(dataY)


    time_step = 100
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, ytest = create_dataset(test_data, time_step)


    X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

    ### Create the Stacked LSTM model
    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=5,batch_size=64,verbose=1)

    last_date = pd.to_datetime("2022-12-12")
    lst_output_1=[]
    lst_output_2=[]

    diff_1 = start_date-last_date
    diff_2 = end_date-last_date
    if int(diff_1.days)>0:
        n_steps=100
        x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        i=0
        while(i<int(diff_1.days)):
            if(len(temp_input)>100):
                #print(temp_input)
                x_input=np.array(temp_input[1:])
    #                 print("{} day input {}".format(i,x_input))
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_steps, 1))
                #print(x_input)
                yhat = model.predict(x_input, verbose=0)
    #                 print("{} day output {}".format(i,yhat))
                temp_input.extend(yhat[0].tolist())
                temp_input=temp_input[1:]
                #print(temp_input)
                lst_output_1.extend(yhat.tolist())
                i=i+1
            else:
                x_input = x_input.reshape((1, n_steps,1))
                yhat = model.predict(x_input, verbose=0)
    #                 print(yhat[0])
                temp_input.extend(yhat[0].tolist())
    #                 print(len(temp_input))
                lst_output_1.extend(yhat.tolist())
                i=i+1


            
    if int(diff_2.days)>0:
        n_steps=100
        x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        i=0
        while(i<int(diff_2.days)):
            if(len(temp_input)>100):
                #print(temp_input)
                x_input=np.array(temp_input[1:])
    #                 print("{} day input {}".format(i,x_input))
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_steps, 1))
                #print(x_input)
                yhat = model.predict(x_input, verbose=0)
    #                 print("{} day output {}".format(i,yhat))
                temp_input.extend(yhat[0].tolist())
                temp_input=temp_input[1:]
                #print(temp_input)
                lst_output_2.extend(yhat.tolist())
                i=i+1
            else:
                x_input = x_input.reshape((1, n_steps,1))
                yhat = model.predict(x_input, verbose=0)
    #                 print(yhat[0])
                temp_input.extend(yhat[0].tolist())
    #                 print(len(temp_input))
                lst_output_2.extend(yhat.tolist())
                i=i+1
    if int(diff_2.days)>0 and int(diff_1.days)>0:     
        invests = scaler.inverse_transform(lst_output_1)[-1]
        returns = scaler.inverse_transform(lst_output_2)[-1]
        Final_Amount = ((returns-invests)*(int(number_of_share)))
    elif int(diff_2.days)>0 and int(diff_1.days)<0: 
        invests = (df[(df['Symbol'] == name_of_stock) & (df['Date'] == start_date)]['Close'].values[0]).reshape(1,-1)
        invests= invests[-1] * (int(number_of_share))
        returns = (scaler.inverse_transform(lst_output_2)[-1])* (int(number_of_share))
        Final_Amount = (returns-invests)
    else:
        invests = (df[(df['Symbol'] == name_of_stock) & (df['Date'] == start_date)]['Close'].values[0]).reshape(1,-1)
        invests= invests[-1] * (int(number_of_share))
        returns = (df[(df['Symbol'] == name_of_stock) & (df['Date'] == end_date)]['Close'].values[0]).reshape(1,-1)
        returns= returns[-1] * (int(number_of_share))
      
        Final_Amount = (returns-invests)


    # In[2]:

    if Final_Amount[0]>0:
        print("Profit: " ,Final_Amount[0])
        return Final_Amount[0]
    else:
        print("Loss: " ,Final_Amount[0])
        return Final_Amount[0]

In [8]:
stockPredictionModel('WIPRO', 30,'2023-1-31', '2025-12-1')

C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '17-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18-12-2018' in DD/MM/YYYY format. Provide format or specify in

Epoch 1/5
9/9 [==============================] - 8s 339ms/step - loss: 0.0555 - val_loss: 0.0422
Epoch 2/5
9/9 [==============================] - 2s 213ms/step - loss: 0.0126 - val_loss: 0.0154
Epoch 3/5
9/9 [==============================] - 2s 182ms/step - loss: 0.0051 - val_loss: 0.0133
Epoch 4/5
9/9 [==============================] - 2s 193ms/step - loss: 0.0022 - val_loss: 0.0027
Epoch 5/5
9/9 [==============================] - 2s 197ms/step - loss: 0.0014 - val_loss: 0.0070
Profit:  18005.90558207663


18005.90558207663

In [9]:
stockPredictionModel('WIPRO', 30,'2022-3-3', '2024-12-1')

C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '17-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18-12-2018' in DD/MM/YYYY format. Provide format or specify in

Epoch 1/5
9/9 [==============================] - 12s 513ms/step - loss: 0.0540 - val_loss: 0.0234
Epoch 2/5
9/9 [==============================] - 2s 197ms/step - loss: 0.0110 - val_loss: 0.0085
Epoch 3/5
9/9 [==============================] - 2s 229ms/step - loss: 0.0057 - val_loss: 0.0049
Epoch 4/5
9/9 [==============================] - 2s 181ms/step - loss: 0.0025 - val_loss: 0.0091
Epoch 5/5
9/9 [==============================] - 2s 177ms/step - loss: 0.0017 - val_loss: 0.0036
Loss:  -6235.080409305876


-6235.080409305876

In [10]:
stockPredictionModel('WIPRO', 30,'2020-2-25', '2022-11-30')

C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '17-12-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18-12-2018' in DD/MM/YYYY format. Provide format or specify in

Epoch 1/5
9/9 [==============================] - 9s 338ms/step - loss: 0.0541 - val_loss: 0.0112
Epoch 2/5
9/9 [==============================] - 1s 167ms/step - loss: 0.0110 - val_loss: 0.0073
Epoch 3/5
9/9 [==============================] - 1s 160ms/step - loss: 0.0036 - val_loss: 0.0080
Epoch 4/5
9/9 [==============================] - 1s 156ms/step - loss: 0.0020 - val_loss: 0.0122
Epoch 5/5
9/9 [==============================] - 1s 165ms/step - loss: 0.0016 - val_loss: 0.0035
Profit:  4883.999639999999


4883.999639999999